In [ ]:
import requests
import sqlite3
import csv
from bs4 import BeautifulSoup

## Task 1

In [ ]:
with open("weather.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

rows = soup.find("table").find("tbody").find_all("tr")

weather_data = []
for row in rows:
    cols = row.find_all("td")
    day = cols[0].text
    temp = int(cols[1].text.replace("°C", ""))  # Convert to integer
    condition = cols[2].text
    weather_data.append({"day": day, "temperature": temp, "condition": condition})

for entry in weather_data:
    print(f"{entry['day']}: {entry['temperature']}°C, {entry['condition']}")

# Finding the hottest day
hottest_day = max(weather_data, key=lambda x: x["temperature"])
print(f"\nHottest day: {hottest_day['day']} with {hottest_day['temperature']}°C")

# Finding all Sunny days
sunny_days = [entry["day"] for entry in weather_data if entry["condition"] == "Sunny"]
print(f"Sunny days: {', '.join(sunny_days)}")

# Compute average temperature
average_temp = sum(entry["temperature"] for entry in weather_data) / len(weather_data)
print(f"Average temperature for the week: {average_temp:.2f}°C")

## Task 2

In [ ]:
conn = sqlite3.connect("jobs.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS jobs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    company TEXT,
    location TEXT,
    description TEXT,
    apply_link TEXT,
    UNIQUE (title, company, location)
)
""")
conn.commit()

url = "https://realpython.github.io/fake-jobs"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

job_listings = soup.find_all("div", class_="card-content")

for job in job_listings:
    title = job.find("h2", class_="title").text.strip()
    company = job.find("h3", class_="company").text.strip()
    location = job.find("p", class_="location").text.strip()
    description = job.find("div", class_="description").text.strip()
    apply_link = job.find("a")["href"]

    cursor.execute("""
    INSERT INTO jobs (title, company, location, description, apply_link) 
    VALUES (?, ?, ?, ?, ?) 
    ON CONFLICT(title, company, location) 
    DO UPDATE SET description=excluded.description, apply_link=excluded.apply_link
    """, (title, company, location, description, apply_link))

conn.commit()
conn.close()

print("Job listings scraped and stored successfully.")